# Perplexity playbook

<p>
Mal Minhas, v0.5<br>
01.08.24
</p>
<p>
<h4>Versions</h4>
<ul>
<li><b>v0.3</b>: 15.01.24. First version</li>
<li><b>v0.4</b>: 02.06.24. Updated for latest Perplexity API</li>
<li><b>v0.5</b>: 01.08.24. Updated again for latest Perplexity API and llama 3.1 support</li>
</ul>
</p>

## 1. Introduction

Perplexity is an intelligent search engine with a chatbot sytle interface that leverages popular AI models to offer comprehensive answers to user inquiries.  It is able to search the internet to provide real-time information via a friendly user experience that includes native mobile apps.  The tool is specifically designed to perform as a research companion providing links to underlying source articles unlike ChatGPT in default mode.  Perplexity comes with an OpenAI-compliant API called [`pplx-api`](https://blog.perplexity.ai/blog/introducing-pplx-api) which was introduced in October 2023.  See [here](https://docs.perplexity.ai/docs/getting-started) for a guide on how to set up an API key and get started with `pplx-api` per the documetnation [here](https://docs.perplexity.ai/).  Perplexity regularly update the model support for this API and at the time of writing the latest and greatest supported model is `llama-3.1`.  The following examples assume you have set up the API key in an environment variable called `PERPLEXITY_API_KEY`.

Supported models are [as follows](https://docs.perplexity.ai/docs/model-cards):
```
Perplexity Models
Model	Parameter Count	Context Length	Model Type
llama-3-sonar-small-32k-online*	8B	28,000	Chat Completion
llama-3-sonar-small-32k-chat*	8B	32,768	Chat Completion
llama-3-sonar-large-32k-online*	70B	28,000	Chat Completion
llama-3-sonar-large-32k-chat*	70B	32,768	Chat Completion
llama-3.1-sonar-small-128k-online	8B	127,072	Chat Completion
llama-3.1-sonar-small-128k-chat	8B	131,072	Chat Completion
llama-3.1-sonar-large-128k-online	70B	127,072	Chat Completion
llama-3.1-sonar-large-128k-chat	70B	131,072	Chat Completion

Open-Source Models
Where possible, we try to match the Hugging Face implementation.
Model	Parameter Count	Context Length	Model Type
llama-3-8b-instruct*	8B	8,192	Chat Completion
llama-3-70b-instruct*	70B	8,192	Chat Completion
mixtral-8x7b-instruct*	8x7B	16,384	Chat Completion
llama-3.1-8b-instruct	8B	131,072	Chat Completion
llama-3.1-70b-instruct	70B	131,072	Chat Completion

* All these models will be deprecated on August 12 2024.
```

The only API function currently supported is on `chat_completions` documented [here](https://docs.perplexity.ai/reference/post_chat_completions).  The next section outlines how to use it.

## 2. Basic Usage

Let's start with a few basics including a utility function to render text for display in Jupyter:

In [1]:
import os
import requests
from IPython.display import display, Markdown

def renderCompletion(text: str):
    display(Markdown(text))
    
API_KEY = os.environ.get('PERPLEXITY_API_KEY')
DEFAULT_MODEL = 'llama-3.1-sonar-small-128k-online'
DEFAULT_TOKENS = 1024
CONCISE_DETAIL = 'Be precise and concise.'
MID_LEVEL_DETAIL = 'Explain in a few sentences.'
GREAT_DETAIL = 'Explain in a lot of detail.'
POLITE_DETAIL = 'You are an artificial intelligence assistant and you need to engage in a helpful, detailed, polite conversation with a user.'
DEFAULT_TEMP = 0.0

The following is a convenience function that wraps the Perplexity API allowing different models to be used and also differing levels of depth of response detail:

In [2]:
def getPerplexityResponse(question:str, model:str=DEFAULT_MODEL, direction:str=MID_LEVEL_DETAIL, max_tokens:int=DEFAULT_TOKENS, temperature:float=DEFAULT_TEMP) -> str:
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": False,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": f"{direction}"
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    r = requests.post(url, json=payload, headers=headers)
    assert(r.status_code == 200)
    return renderCompletion(r.json().get("choices")[0].get("message").get("content"))

Let's test the function on an example query using the default `llama-3.1-sonar-small-128k-online` model and then `llama-3.1-8b-instruct`.  Note the differences in response timing:

In [3]:
%%time

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
getPerplexityResponse(prompt)

To get your broken washing machine repaired in Southampton, UK, you can contact one of the following appliance repair services:

1. **Hedge End Domestics**: They offer a same or next-day repair service for various appliances, including washing machines. You can call them at 01489 782 072 or visit their website for more information.

2. **Southampton Appliance Repairs**: This service has over 15 years of experience and works with major manufacturers. They offer a fast, professional, and guaranteed repair service. You can call them at 02380 012208 or visit their website for more details.

3. **Go Assist**: They provide affordable fixed-price repairs for washing machines and have a nationwide network of reliable engineers. You can book a repair online or call them at 0333 733 1234.

These services can help you get your washing machine repaired efficiently and effectively.

CPU times: user 12 ms, sys: 3.41 ms, total: 15.4 ms
Wall time: 2.99 s


In [4]:
%%time

getPerplexityResponse(prompt, model='llama-3.1-8b-instruct')

Sorry to hear that your washing machine is broken!

In the UK, you'll want to call a professional appliance repair service to fix your washing machine. You can search online for "washing machine repair Southamption" or "appliance repair Southamption" to find local companies that offer this service.

Typically, you'll want to contact a:

* Appliance repair engineer (also known as a white goods repair engineer)
* Domestic appliance repair service
* Washing machine repair specialist

These professionals will send someone to assess and fix your washing machine at your home. You can also check online review sites such as Trustpilot or Google Reviews to find a reputable and reliable repair service in your area.

CPU times: user 9.93 ms, sys: 3.38 ms, total: 13.3 ms
Wall time: 1.31 s


In [5]:
%%time

getPerplexityResponse(prompt, model='llama-3.1-8b-instruct', direction=POLITE_DETAIL)

I'm so sorry to hear that your washing machine is broken Don't worry, I'm here to help you get it fixed.

In the UK, the trade you'll need to call is an appliance repair technician, specifically a washing machine repair specialist. They will be able to diagnose the issue and fix it for you.

Here are a few options to consider:

1. **Local appliance repair services**: You can search online for appliance repair services in Southampton. Websites like Rated People, MyBuilder, or Checkatrade can connect you with local, vetted tradespeople who specialize in washing machine repairs.
2. **Manufacturer's authorized repair service**: If your washing machine is still under warranty or you want to use the manufacturer's authorized repair service, you can contact the manufacturer's customer support to find a local authorized repair center.
3. **Independent repair shops**: You can also search for independent repair shops in Southampton that specialize in appliance repairs. They may offer more competitive pricing and flexible appointment times.
4. **National repair companies**: Companies like AO Repair, Currys PC World Repair, or Hughes Repair offer appliance repair services across the UK, including in Southampton.

Before calling a repair service, it's a good idea to:

* Check your warranty to see if it's still valid and what's covered.
* Gather information about the issue with your washing machine, such as error codes or symptoms.
* Get quotes from multiple repair services to compare prices and services.

When you call a repair service, be sure to ask about their:

* Experience with washing machine repairs
* Availability and appointment times
* Pricing and any additional costs
* Warranty or guarantee on their work

I hope this helps you get your washing machine fixed quickly and efficiently

CPU times: user 8.51 ms, sys: 2.76 ms, total: 11.3 ms
Wall time: 2.54 s


In [6]:
%%time

getPerplexityResponse(prompt, model='llama-3.1-sonar-small-128k-online', direction=POLITE_DETAIL)

I understand you have a broken washing machine and live in Southampton, UK. There are several options you can consider to get it repaired:

1. **Domestic Appliance Repair Services**:
   - **Hedge End Domestics**: This is a family-run business with over 35 years of experience in domestic appliance repairs. They offer same or next-day service and carry spare parts for major brands. You can contact them at 01489 782 072 or visit their website for more information.

2. **Southampton Appliance Repairs**: This service has over 15 years of experience and works with major manufacturers. They offer a fast, professional, and guaranteed repair service. You can call them at 02380 012208 or visit their website for more details.

3. **Go Assist**: This company offers affordable fixed prices for washing machine repairs and has a nationwide network of reliable engineers. They provide same-day or next-day services and are available in Southampton. You can book online or call them at 0333 733 1234.

4. **NAC Repair**: While primarily focused on domestic appliances, they also offer repair services in Southampton. They have a network of experienced engineers and provide a repair guarantee. You can contact them at 0333 016 9622 or visit their website for more information.

Each of these services offers a reliable and efficient way to get your washing machine repaired. You can choose the one that best fits your needs based on their availability, pricing, and reputation.

CPU times: user 9.98 ms, sys: 3.11 ms, total: 13.1 ms
Wall time: 3.92 s


Let's see how it handles a query with a more detailed response:

In [7]:
%%time

prompt = """"I have a broken garden gate which is no longer closing properly.  
What trade do I need to call to fix it?"""
getPerplexityResponse(prompt, model='llama-3.1-sonar-small-128k-online', direction=GREAT_DETAIL)

To fix a broken garden gate that is no longer closing properly, you would typically need to call a carpenter or a handyman. These professionals are skilled in working with wood and metal, which are common materials used in garden gates. They can assess the issue, identify the problem, and perform the necessary repairs to ensure the gate functions correctly again.

Here are some steps you might follow to fix a sagging or misaligned garden gate:

1. **Identify the Problem**: Determine the cause of the gate's malfunction. Common issues include sagging, misalignment, worn-out hinges, or damaged latch mechanisms.

2. **Assess the Gate**: Inspect the gate for any visible damage or wear. Check if the hinges are loose or if the latch is not engaging properly.

3. **Gather Tools and Materials**: Collect the necessary tools and materials, such as a screwdriver, adjustable wrench, level, tape measure, hammer, saw, pliers, lubricant (like WD-40), and replacement parts (hinges, latches, screws).

4. **Support the Gate**: If the gate is sagging, support it with temporary bracing to prevent further damage while you work on it.

5. **Remove the Gate**: Unscrew and lift the gate off the hinges to inspect for damage. Clean the hinge area to ensure proper alignment.

6. **Repair Broken Parts**: Fix any broken parts, such as hinges or the latch mechanism. Replace damaged wood or metal components if necessary.

7. **Realign and Reattach**: Align the gate properly and mark the hinge positions. Securely reattach the hinges and latch, ensuring they are tightened correctly.

8. **Test the Gate**: Open and close the gate several times to check its movement and ensure it is level. Test the latch to make sure it engages properly.

9. **Make Adjustments**: If necessary, make adjustments to the gate's alignment or tighten any loose screws. Apply lubricant to moving parts to keep them working smoothly.

By following these steps and calling a carpenter or handyman if needed, you can effectively fix a broken garden gate that is no longer closing properly.

CPU times: user 9.59 ms, sys: 4.77 ms, total: 14.4 ms
Wall time: 4.6 s


## 3. Streaming interface

This is an outline of how to build a streaming interface for Perplexity.  More work needs to be done to flesh it out.

In [8]:
%%time

def getPerplexityStreamingResponse(question, model=DEFAULT_MODEL, direction=MID_LEVEL_DETAIL, max_tokens=DEFAULT_TOKENS, temperature=DEFAULT_TEMP):
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": True,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": direction,
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    print('start')
    response_stream = requests.post(url, json=payload, headers=headers)
    print('end')
    for response in response_stream:
        print(len(response))
    #return response.get("choices")[0].get("message").get("content")

prompt = """I have a broken washing machine and I live in Southamption in the UK. 
What can I do to get it mended?  What trade do I need to call?"""
#getPerplexityStreamingResponse(prompt, direction=GREAT_DETAIL)

CPU times: user 4 μs, sys: 1e+03 ns, total: 5 μs
Wall time: 7.87 μs
